# Heroes of Pymoli

* Observed Trend 1:  There are significantly more males playing the game than females
* Observed Trend 2:  Those that report their gender as "Other / non-disclosed" spend more per purchase on average than either those that report their gender as male or female.
* Observed Trend 3:  People 20-24 spend the most and have the most transactions; however, people 8-12 have the highest average spend

In [123]:
# dependencies
import pandas as pd
import numpy as np

In [124]:
# assigning path to JSON data
jsonpath = input("What is the name / path of the JSON file that you wish to analyze?")

In [174]:
#creating a data frame from purchase data
purchase_data_df = pd.read_json(jsonpath)
purchase_data_df.head()

,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59


* Total Number of Players

In [127]:
#find total number of players
tot_players = purchase_data_df['SN'].nunique()
tot_players

573

## Purchasing Analysis (Total)
* Number of Unique Items
* Average Purchase Price
* Total Number of Purchases
* Total Revenue

In [128]:
# Find summary statistics about the purchases

#total unique items
tot_items = purchase_data_df['Item ID'].nunique()

#average price of items
avg_price = purchase_data_df['Price'].mean()

#total purchases
tot_purchases = len(purchase_data_df)

#total revenue
tot_revenue = purchase_data_df['Price'].sum()

In [129]:
# Create a dataframe collecting the purchase analysis data
purchase_analysis_df = pd.DataFrame(data = [tot_items, avg_price, tot_purchases, tot_revenue], index = ['Total Items', 'Average Price', 'Total Purchases', 'Total Revenue'], columns = ["Data"])
purchase_analysis_df.head()

,Data
Total Items,183.000000
Average Price,2.931192
Total Purchases,780.000000
Total Revenue,2286.330000


## Gender Demographics
* Percentage and Count of Male Players
* Percentage and Count of Female Players
* Percentage and Count of Other / Non-Disclosed

In [212]:
# Create a DataFrame with a purchase count by gender
gender_df = purchase_data_df[['Item ID', "Gender"]].groupby(by = "Gender").count()
gender_df.rename(columns = {"Item ID": "Purchase Count"}, inplace = True)

# Create a separate dataframe to get the unique counts of gender
gender_unique_count = purchase_data_df.drop_duplicates(['SN'], keep = "last")
gender_df['Unique Count'] = gender_unique_count[['SN', "Gender"]].groupby(by = "Gender").count()
gender_df

,Purchase Count,Unique Count
Gender,,
Female,136,100
Male,633,465
Other / Non-Disclosed,11,8


In [138]:
# Find statistics by gender
males_count = gender_df.loc['Male'][0]
males_percent = males_count / tot_purchases
females_count = gender_df.loc['Female'][0]
females_percent = females_count / tot_purchases
other_count = gender_df.loc['Other / Non-Disclosed'][0]
other_percent = other_count / tot_purchases

In [218]:
# create a dictionary into which the statistics can later be placed into a dataframe
my_dict = {"Purchase Count": {"males": males_count, "females": females_count, "others": other_count}, 
           "Percent": {"females": females_percent, "males": males_percent, "others": other_count}}

In [219]:
# create a dataframe from the dictionary
gender_demographics_df = pd.DataFrame(data = my_dict)
gender_demographics_df.head()

,Percent,Purchase Count
females,0.174359,136
males,0.811538,633
others,11.000000,11


## The below each broken by gender
  * Purchase Count
  * Average Purchase Price
  * Total Purchase Value
  * Normalized Totals

In [220]:
# create a DataFrame on the index of the gender data
gender_metrics = pd.DataFrame(index = gender_df.index)

In [222]:
# create a new column in the DataFrame and assign the counts by age.
gender_metrics['Purchase Count'] = gender_df['Purchase Count']

In [223]:
# create a new column in the DataFrame and put in average price data by gender
gender_metrics['Average Purchase Price'] = purchase_data_df.groupby(by = "Gender").mean()['Price']

In [226]:
# create a new column in the DataFrame and put in the total price by gender
gender_metrics['Total Purchase Value'] = purchase_data_df.groupby(by = "Gender").sum()['Price']

In [227]:
# create normalized average
gender_metrics['Normalized Average'] = gender_metrics['Total Purchase Value'] / gender_df["Unique Count"]
gender_metrics

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Average
Gender,,,,
Female,136,2.815515,382.91,3.829100
Male,633,2.950521,1867.68,4.016516
Other / Non-Disclosed,11,3.249091,35.74,4.467500


## Age Demographics

* The below each broken into bins of 4 years (i.e. &lt;10, 10-14, 15-19, etc.) 
* Purchase Count
* Average Purchase Price
* Total Purchase Value

In [179]:
# create bins based on the shape of the data for age
bins = np.arange(0, purchase_data_df['Age'].max(), 4)
labels = ["0-4", "4-8", "8-12", "12-16", "16-20", "20-24", "28-32", "32-36", "36-40", "40-44", "44+"]

In [230]:
# insert the bins into the dataframe
purchase_data_df['Age Group'] = pd.cut(purchase_data_df['Age'], bins = bins, labels = labels)

In [183]:
# group the data by the bins created by the ages in the data set & create a new dataframe 
#first containing the count of purchases by age group
age_demo_df = purchase_data_df[['Age Group', 'Price']].groupby('Age Group').count()
age_demo_df.rename(columns = {"Price": "Purchase Count"}, inplace = True)
age_demo_df.head()

,Purchase Count
Age Group,
0-4,0
4-8,22
8-12,24
12-16,87
16-20,161


In [184]:
#Create a new column with the total purchase by age group
age_demo_df['Total Purchase Value'] = purchase_data_df[['Age Group', 'Price']].groupby('Age Group').sum()

In [234]:
# Create a separate dataframe to get the unique counts of age
age_unique_count = purchase_data_df.drop_duplicates(['SN'], keep = "last")
age_demo_df['Unique Count'] = age_unique_count[['SN', "Age Group"]].groupby(by = "Age Group").count()

,Purchase Count,Total Purchase Value,Average Purchase Price,Unique Count
Age Group,,,,
0-4,0,NaN,NaN,0
4-8,22,61.34,2.788182,13
8-12,24,81.25,3.385417,18
12-16,87,238.89,2.745862,64
16-20,161,468.03,2.907019,120
20-24,238,696.09,2.924748,186
28-32,104,309.37,2.974712,75
32-36,66,202.09,3.061970,44
36-40,38,113.28,2.981053,28


In [233]:
#Create a new column with the average purchase price by age group
age_demo_df['Average Purchase Price'] = age_demo_df['Total Purchase Value'] / age_demo_df['Purchase Count']

In [235]:
age_demo_df['Normalized Average'] = age_demo_df['Total Purchase Value'] / age_demo_df['Unique Count']
age_demo_df

,Purchase Count,Total Purchase Value,Average Purchase Price,Unique Count,Normalized Average
Age Group,,,,,
0-4,0,NaN,NaN,0,NaN
4-8,22,61.34,2.788182,13,4.718462
8-12,24,81.25,3.385417,18,4.513889
12-16,87,238.89,2.745862,64,3.732656
16-20,161,468.03,2.907019,120,3.900250
20-24,238,696.09,2.924748,186,3.742419
28-32,104,309.37,2.974712,75,4.124933
32-36,66,202.09,3.061970,44,4.592955
36-40,38,113.28,2.981053,28,4.045714


**Top Spenders**

* Identify the the top 5 spenders in the game by total purchase value, then list (in a table):
  * SN
  * Purchase Count
  * Average Purchase Price
  * Total Purchase Value

In [186]:
#Create a dataframe with the purchase count by the players
players_df = purchase_data_df[['SN', "Price"]].groupby('SN').count()
players_df.rename(columns = {"Price": "Purchase Count"}, inplace = True)
players_df.head()

,Purchase Count
SN,
Adairialis76,1
Aduephos78,3
Aeduera68,3
Aela49,1
Aela59,1


In [187]:
#  Add in teh total purchase value for each player
players_df['Total Purchase Value'] = purchase_data_df[['SN', 'Price']].groupby('SN').sum()
players_df.head()

,Purchase Count,Total Purchase Value
SN,,
Adairialis76,1,2.46
Aduephos78,3,6.70
Aeduera68,3,5.80
Aela49,1,2.46
Aela59,1,1.27


In [188]:
#Find the average purchase price
players_df['Average Purchase Price'] = players_df['Total Purchase Value'] / players_df['Purchase Count']

In [189]:
#Find the top five players based on total purchase value
players_df.sort_values(by = "Total Purchase Value", ascending = False, inplace = True)
top_5_players = players_df.iloc[0:5 , :]
top_5_players

,Purchase Count,Total Purchase Value,Average Purchase Price
SN,,,
Undirrala66,5,17.06,3.412000
Saedue76,4,13.56,3.390000
Mindimnya67,4,12.74,3.185000
Haellysu29,3,12.73,4.243333
Eoda93,3,11.58,3.860000


**Most Popular Items**

* Identify the 5 most popular items by purchase count, then list (in a table):
  * Item ID
  * Item Name
  * Purchase Count
  * Item Price
  * Total Purchase Value

In [190]:
#create a dataframe that contains the item information and initially creates a purchase count for all items
items_df = purchase_data_df[['Item ID', "Item Name", "Price"]].groupby(['Item ID', 'Item Name']).count()
items_df.rename(columns = {"Price": "Purchase Count"}, inplace = True)
items_df.head()

,,Purchase Count
Item ID,Item Name,
0,Splinter,1
1,Crucifer,4
2,Verdict,1
3,Phantomlight,1
4,Bloodlord's Fetish,1


In [191]:
#Add the total purchase value for each item to the dataframe
items_df["Purchase Value"] = purchase_data_df[['Item ID', "Item Name", "Price"]].groupby(['Item ID', 'Item Name']).sum()

In [192]:
# Find the average purchase value of the items and assign to a column into the dataframe
items_df["Average Purchase Value"] = items_df['Purchase Value'] / items_df['Purchase Count']

In [193]:
#Sort the data frame according to the Purchase Value column and find the most popular items by value
items_df.sort_values(by = "Purchase Value", inplace = True, ascending = False)
most_popular_items_by_value = items_df.iloc[ :5, :]
most_popular_items_by_value

,,Purchase Count,Purchase Value,Average Purchase Value
Item ID,Item Name,,,
34,Retribution Axe,9,37.26,4.14
115,Spectral Diamond Doomblade,7,29.75,4.25
32,Orenmir,6,29.70,4.95
103,Singed Scalpel,6,29.22,4.87
107,"Splitter, Foe Of Subtlety",8,28.88,3.61


In [194]:
#Sort the data frame according to the Purchase Count column and find the most popular items by value
items_df.sort_values(by = "Purchase Count", inplace = True, ascending = False)
most_popular_items_by_count = items_df.iloc[ :5, :]
most_popular_items_by_count

,,Purchase Count,Purchase Value,Average Purchase Value
Item ID,Item Name,,,
39,"Betrayal, Whisper of Grieving Widows",11,25.85,2.35
84,Arcane Gem,11,24.53,2.23
34,Retribution Axe,9,37.26,4.14
13,Serenity,9,13.41,1.49
31,Trickster,9,18.63,2.07
